In [4]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
from math import ceil
from image import *
from image_utils import *
from stitch import *

In [5]:
rm ../data/nevada/panorama.jpg

rm: cannot remove '../data/nevada/panorama.jpg': No such file or directory


In [6]:
images = read_images("../data/nevada/")

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
for img in images:
    img.show()

In [ ]:
def warp_two_images(img1, img2, h):
    """Warps the given image img1 to img2 using homographt matrix h.

    Arguments:
        img1 {np.ndarray} -- Image to be warped
        img2 {np.ndarray} -- Image to which img1 has to be warped
        h {np.ndarray} -- Homography matrix

    Returns:
        np.ndarray -- Image resulting after warping img1 to img2
    """

    # Find the shape
    y1, x1 = img1.shape[:2]
    y2, x2 = img2.shape[:2]

    # Get the corners of the images
    pts1 = np.array([[0, 0], [0, y1], [x1, y1], [x1, 0]],
                    dtype=np.float32).reshape(-1, 1, 2)
    pts2 = np.array([[0, 0], [0, y2], [x2, y2], [x2, 0]],
                    dtype=np.float32).reshape(-1, 1, 2)

    # Get the points after applying homography
    pts2_ = cv2.perspectiveTransform(pts2, h)
    pts = np.concatenate((pts1, pts2_), axis=0)

    # Round off the points
    [xmin, ymin] = np.array(pts.min(axis=0).flatten() - 0.5, dtype=np.int32)
    [xmax, ymax] = np.array(pts.max(axis=0).flatten() + 0.5, dtype=np.int32)

    # Compute the translation homography matrix
    warped_img = [-xmin, -ymin]
    h_dash = np.array(
        [[1, 0, warped_img[0]], [0, 1, warped_img[1]], [0, 0, 1]])  # translate

    # Use the translation homography matrix to fit bot img1 and img2 on mossiac
    res1 = cv2.warpPerspective(img2, h_dash.dot(h), (xmax-xmin, ymax-ymin))
    res2 = np.zeros_like(res1)
    res2[warped_img[1]:y1+warped_img[1],warped_img[0]:x1+warped_img[0]] = img1
    
    return res1, res2


In [ ]:
matches = knn_matcher(images[0].descriptors, images[1].descriptors)

# Compute the homography using ransac
h = ransac(matches, images[0].keypoints, images[1].keypoints,
           sample_points=10, iterations=10000, check=False)

res1, res2 = warp_two_images(images[1].img, images[0].img, h)

In [ ]:
def blender(res1, res2, show_unblended = True):
    overlapping = np.logical_and(res1 >0 , res2>0)

    temp = res1.copy()
    temp[~overlapping] = 0
    non_zeros = np.where(temp > 0)[1]
    y_min, y_max = non_zeros.min(), non_zeros.max()

    res1 = np.divide(res1, 255, dtype=np.float)
    res2 = np.divide(res2, 255, dtype=np.float)
    
    mask1_row = np.ones_like(res1[0], dtype=np.float32)
    mask1_row[y_min:y_max] = np.repeat(np.linspace(1, 0, num = y_max - y_min), 3).reshape(-1,3)
    mask1 = np.array([mask1_row for i in range(res1.shape[0])])


    mask2_row = np.ones_like(res1[0], dtype=np.float32)
    mask2_row[y_min:y_max] = np.repeat(np.linspace(0, 1, num = y_max - y_min), 3).reshape(-1,3)
    mask2 = np.array([mask2_row for i in range(res1.shape[0])])

    if show_unblended:
        temp_img = res1 + res2
        temp_img[overlapping] = res1[overlapping]
        show_image(temp_img)
    return np.multiply(res1, mask2) + np.multiply(res2, mask1)

In [ ]:
blended = blender(res1, res2)
show_image(blended)

In [ ]:
blended[blended >= 1] = 1
blended = np.int32(blended * 255)

In [ ]:
cv2.cvtColor(blended, cv2.COLOR_BGR2GRAY)

In [ ]:
Image(blended)

In [ ]:
blended_img = Image(blended)
matches = knn_matcher(blended_img.descriptors, images[2].descriptors)

# Compute the homography using ransac
h = ransac(matches, blended_img.keypoints, images[2].keypoints,
           sample_points=10, iterations=10000, check=False)

res1, res2 = warp_two_images(images[2].img, blended_img.mg, h)

In [ ]:
# show_image(res2)
# show_image(mask1)
# show_image(res1)
# show_image(mask2)
# show_image(np.multiply(res1,mask2))
# show_image(np.multiply(res2,mask1))
# show_image(np.multiply(res1, mask2) + np.multiply(res2, mask1))